In [ ]:
from selenium import webdriver
import time

class Footsal_Reservation():

    #유캠퍼스 로그인
    def uCampus_Login(self, driver, User_id, User_pw):
        driver.get("YOURSITE")
        driver.find_element_by_css_selector("tbody")
        elem = driver.find_element_by_name("member_no")
        elem.send_keys(User_id)
        elem = driver.find_element_by_name("password")
        elem.send_keys(User_pw)
        elem.submit()

    #장소사용 페이지를 검색한다.   
    def search_Page(self, drive, date):
        '''
        place_code = 1 -> 풋살장
        pace_code = 3 -> 복지관 학생식당
        '''
        driver.find_element_by_xpath("//select[@name='place_code']/option[@value='1']").click() #풋살장 선택
        elem = driver.find_element_by_xpath("//input[@name='rent_date']") #대여 날짜
        elem.send_keys(date)
        elem = driver.find_element_by_xpath("//input[@id='btn_submit']")#제출
        elem.click()


    # 예약구분에서 가능 시간대를 고른다.(select_Time)
    '''
    2. 10:30 ~ 12:00
    3. 12:00 ~ 13:30
    4. 13:30 ~ 15:00
    5. 15:00 ~ 16:30
    6. 16:30 ~ 18:00
    7. 18:00 ~ 19:30
    8. 19:30 ~ 21:00
    '''

    #원하는 시간대를 신청한다.    
    def select_Time(self, driver,time_Num):
        elem = driver.find_element_by_xpath("//input[@id='btn_submit']")
        elem = driver.find_element_by_xpath("//input[@onclick='javascript:rent_proc({});']".format(time_Num))# time_Num 변수에 원하는 시간대를 넣는다.
        elem.click()

    #예약페이지    
    def reservation_Page_Detail(self, driver):
        elem = driver.find_element_by_xpath("//input[@id='renter_gubun3']") #기타 선택
        elem.click()

        elem = driver.find_element_by_xpath("//input[@name='etc_rent_name']")#단체명
        elem.send_keys('테스트')

        elem = driver.find_element_by_xpath("//input[@id='hp_no']")#폰 번호
        elem.send_keys('010-7331-4120')

        elem = driver.find_element_by_xpath("//input[@id='man_count']")#참가 인원
        elem.send_keys('12')

        elem = driver.find_element_by_xpath("//input[@id='purpose']")#행사명(사용목적)
        elem.send_keys('테스트 정기 풋살 모임 입니다.')

    #최종 제출
    def done_Submit(self, driver):    
        elem = driver.find_element_by_xpath("//input[@id='btn_input_1']")# 신청 버튼
        elem.click()
        alter1 = driver.switch_to_alert()
        alter1.accept()
        time.sleep(1)
        alter1 = driver.switch_to_alert()
        alter1.accept()
    
if __name__ == "__main__":
    
    start = time.time()
    
    driver = webdriver.Firefox()
    #driver = webdriver.PhantomJS()
        
    obj = Footsal_Reservation()
    obj.uCampus_Login(driver,"USERID","USERPW")
    time.sleep(1)
    
    cnt =0
    cnt_reserver = 0
    
    #,"27","27","28","28"
    for date in [ "25","25","26","26","24","24"]:

        if cnt == 0:
            
            if cnt_reserver > 0:
                driver.get("YOURSITE")
                
            obj.search_Page(driver,"2017-07-{}".format(date))
            obj.select_Time(driver,1)
            cnt+=1
            cnt_reserver+=1
            
        elif cnt == 1:
            time.sleep(1)
            obj.select_Time(driver,2)
            cnt=0
            
        obj.reservation_Page_Detail(driver)
        obj.done_Submit(driver)

    end = time.time() - start
    print(end)

In [3]:
import requests
from bs4 import BeautifulSoup
import json

import time
import re 
import smtplib



def send_email():

    TO =['YOUREMAIL']
    SUBJECT ="장소사용 신청 관련 공지사항이 올라왔습니다!"
    TEXT = "장소사용 신청 관련 공지사항이 올라왔습니다!\nhttps://www.kw.ac.kr/ko/life/notice.do"

    # Gmail Sign In
    gmail_sender = 'YOUREMAIL'
    gmail_passwd = 'YOURPASSWORD'

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.login(gmail_sender, gmail_passwd)

    
    for man in TO:
        BODY = '\r\n'.join(['To: {}'.format(man),
                            'From: {}'.format(gmail_sender),
                            'Subject: {}'.format(SUBJECT),
                            '', TEXT]).encode('utf-8')
        try:
            server.sendmail(gmail_sender, [man], BODY)
            print ('email sent')
        except:
            print ('error sending mail')        
    

    server.quit()

def kw_notice():
    
    url = "YOURHOMEPAGE"
    u_a = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.82 Safari/537.36"


    response = requests.get(url, headers={"USER-AGENT":u_a})
    soup = BeautifulSoup(response.text, 'html.parser')
    
    div_soup = soup.find("div",{"class":"list-box"})
    a_soup = div_soup.find_all("a")
    p_soup = div_soup.find_all("p",{"class":"info"})

    message="공지사항 보기 : http://www.kw.ac.kr/ko/life/notice.do\n\n"
    cnt = 0
    
    for ele in a_soup[:25]:
        if len(message) <= 2000:
            if "\n" in ele.text:
                text = ele.text.replace("\t","")
                text = text.replace("신규게시글","")
                text = text.replace("\n","")
            message += str(cnt+1)+". " +  text + "\n" + p_soup[cnt].text + "\n\n"
            cnt+=1
        else:
            break
            
   
    data = "장소사용 신청 안내(2017년 7월)"
    
    if data in message:
        return True
    else:
        return False

if __name__ == "__main__":
    if kw_notice():
        send_email()
        print("True")
    else:
        print("False")